In [43]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re
import sys
from datetime import datetime
import time
import numpy as np
import json

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.chrome.options import Options


In [2]:
# def create_url():
#     departure_city = input('three letter city code departing from')
#     depart_date = input('what month do you want to leave, YYMM format')
#     return_date = input('what month do you want to return, YYMM format')

# depart_city = 'den'
# depart_date = '1811'
# return_date = '1811'

# url = 'https://www.skyscanner.com/transport/flights-from/{}/?adults=1&children=0&adultsv2=1&childrenv2=&infants=0&cabinclass=economy&rtn=1&preferdirects=false&outboundaltsenabled=false&inboundaltsenabled=false&oym={}&iym={}'.format(depart_city, depart_date, return_date)
# url


'https://www.skyscanner.com/transport/flights-from/den/?adults=1&children=0&adultsv2=1&childrenv2=&infants=0&cabinclass=economy&rtn=1&preferdirects=false&outboundaltsenabled=false&inboundaltsenabled=false&oym=1811&iym=1811'

In [93]:
#build url
# start = 'DEN'
# month = '2018-11'
# end = 'US'

start = input('FROM city (3 letter code)')
month = input('Departure month (YYYY-MM)')
destination = input('TO country (US = Unite States, MX = Mexico, CA = Canada)')

api_url = 'https://www.skyscanner.com/g/browseservice/dataservices/browse/v3/bvweb/US/USD/en-US/destinations/{}/{}/{}/?profile=minimalcityrollupwithnamesv2&include=image&apikey=8aa374f4e28e4664bf268f850f767535'.format(start, end, month)

api_url

FROM city (3 letter code)DEN
Departure month (YYYY-MM)2018-12
TO country (US = Unite States, MX = Mexico, CA = Canada)US


'https://www.skyscanner.com/g/browseservice/dataservices/browse/v3/bvweb/US/USD/en-US/destinations/DEN/US/2018-12/?profile=minimalcityrollupwithnamesv2&include=image&apikey=8aa374f4e28e4664bf268f850f767535'

In [52]:
response = requests.get(api_url).json()
response

{'Metadata': {'ExperimentAssignments': [],
  'MachineName': 'FLPVWEW1IDS001',
  'PriceSourceId': 'taps-taps',
  'Properties': [{'Name': 'Browse Service All Results', 'Value': 348},
   {'Name': 'Browse Service Average Quote Age', 'Value': 263579},
   {'Name': 'Browse Service Node', 'Value': 'FLPVWEW1IDS001'},
   {'Name': 'Browse Service Priced Results', 'Value': 214},
   {'Name': 'Browse Service Search Duration', 'Value': 0.0163},
   {'Name': 'Browse Service Version', 'Value': '1.0.3125.0'},
   {'Name': 'Browse Service Zone', 'Value': 'Unknown'}],
  'Revision': '1.0.3125.0'},
 'PlacePrices': [{'Id': 'ABEA', 'IndirectPrice': 142, 'Name': 'Allentown'},
  {'Id': 'ABIA', 'IndirectPrice': 0, 'Name': 'Abilene'},
  {'DirectPrice': 49,
   'Id': 'ABQA',
   'ImageUrl': 'https://content.skyscnr.com/12de2ac3717cab3632d4e3818a6850d9/GettyImages-477728100.jpg',
   'Name': 'Albuquerque'},
  {'Id': 'ABRA', 'IndirectPrice': 0, 'Name': 'Aberdeen'},
  {'Id': 'ABYA', 'IndirectPrice': 0, 'Name': 'Albany'},


In [57]:
response.keys()

dict_keys(['PlacePrices', 'Metadata'])

In [64]:
places = response['PlacePrices']
places

[{'Id': 'ABEA', 'IndirectPrice': 142, 'Name': 'Allentown'},
 {'Id': 'ABIA', 'IndirectPrice': 0, 'Name': 'Abilene'},
 {'DirectPrice': 49,
  'Id': 'ABQA',
  'ImageUrl': 'https://content.skyscnr.com/12de2ac3717cab3632d4e3818a6850d9/GettyImages-477728100.jpg',
  'Name': 'Albuquerque'},
 {'Id': 'ABRA', 'IndirectPrice': 0, 'Name': 'Aberdeen'},
 {'Id': 'ABYA', 'IndirectPrice': 0, 'Name': 'Albany'},
 {'Id': 'ACKA',
  'ImageUrl': 'https://content.skyscnr.com/510bb2de805c6564c92d79e4b0310cc3/GettyImages-528462775.jpg',
  'IndirectPrice': 0,
  'Name': 'Nantucket'},
 {'Id': 'ACTA', 'IndirectPrice': 1172, 'Name': 'Waco'},
 {'Id': 'ACVA',
  'ImageUrl': 'https://content.skyscnr.com/d28ea6c2e0047c678a5092050e8f3449/GettyImages-533132753.jpg',
  'IndirectPrice': 0,
  'Name': 'Arcata'},
 {'Id': 'AIYA',
  'ImageUrl': 'https://content.skyscnr.com/c69a495f16e86dbf1584fd1330a2137f/GettyImages-529372053.jpg',
  'IndirectPrice': 0,
  'Name': 'Atlantic City'},
 {'Id': 'AEXA',
  'ImageUrl': 'https://content.sky

In [88]:
df = pd.DataFrame(places)
df = df[['Name', 'DirectPrice', 'IndirectPrice']]
df

,Name,DirectPrice,IndirectPrice
0,Allentown,NaN,142.0
1,Abilene,NaN,0.0
2,Albuquerque,49.0,NaN
3,Aberdeen,NaN,0.0
4,Albany,NaN,0.0
5,Nantucket,NaN,0.0
6,Waco,NaN,1172.0
7,Arcata,NaN,0.0
8,Atlantic City,NaN,0.0
9,Alexandria,NaN,0.0


In [90]:
#merge direct and indirect price columns
df['price'] =df['DirectPrice'].fillna(df['IndirectPrice'])
df = df[['Name', 'price']]
df

,Name,price
0,Allentown,142.0
1,Abilene,0.0
2,Albuquerque,49.0
3,Aberdeen,0.0
4,Albany,0.0
5,Nantucket,0.0
6,Waco,1172.0
7,Arcata,0.0
8,Atlantic City,0.0
9,Alexandria,0.0


In [106]:
destinations = df.query('price > 0').copy()
destinations = destinations.sort_values('price')
destinations

,Name,price
318,Tucson,27.0
83,Cincinnati,34.0
95,Des Moines,34.0
86,Dallas,35.0
146,Houston,44.0
232,Omaha,44.0
298,Salt Lake City,44.0
317,Tulsa,44.0
175,Las Vegas,44.0
201,Chicago,46.0


In [109]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///C:\\Users\\snowr\\ds_python1\\flight data\\flights.db', echo=False)


In [110]:
destinations.to_sql("flight_prices", con=engine, if_exists='replace')